In [1]:
import re 
import pandas as pd
from nltk.corpus import stopwords
import pymorphy3
import nltk

In [2]:
df_articles = pd.read_excel('date_random.xlsx')
df_articles.head()

,date,title,link,annotation,authors,article_level,text
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,NaN,﻿GEOLOGICAL AND MINERALOGICAL SCIENCES\nTHE PR...
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",NaN,﻿TECHNICAL SCIENCES\nUNDERSTANDING AND DESIGNI...
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",NaN,﻿DEVELOPMENT OF AUTHENTICATION MODEL FOR CLOUD...
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,NaN,﻿DEVELOPMENT OF A MODEL OF INFORMATION FLOWS I...
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,NaN,﻿AUTOMATED CONTROL SYSTEM OF THE SOLAR AIR MAN...


#### 1. Проверка пустых значений

In [3]:
df_articles.shape

(2340, 7)

In [4]:
nan_count = df_articles['date'].isna().sum().sum()
print('date: ', nan_count)
nan_count = df_articles['title'].isna().sum().sum()
print('title: ', nan_count)
nan_count = df_articles['link'].isna().sum().sum()
print('link: ', nan_count)
nan_count = df_articles['annotation'].isna().sum().sum()
print('annotation: ', nan_count)
nan_count = df_articles['authors'].isna().sum().sum()
print('authors: ', nan_count)
nan_count = df_articles['article_level'].isna().sum().sum()
print('article_level: ', nan_count)
nan_count = df_articles['text'].isna().sum().sum()
print('text: ', nan_count)

date:  0
title:  0
link:  0
annotation:  240
authors:  11
article_level:  1198
text:  0


Очень много пустых аннотаций. Их можно заполнить прочерками, а в будущем попробовать извлечь ключевые слова или важную информацию из текстов статей и дозаполнить недостающие значения.

Строки где пропущены авторы удаляем, т.к будущая система рекомендаций не сможет работать без указания конкретных людей.

article_level заполним прочерками, т.к информация об уровне статьи не критически важна для выделения интереса к научной области. 

Строки где пустой текст удаляем целиком, т.к без текста статьи тоже что-то исследовать сложно, и таких строк мало - всего 15.

#### 2. Обработка пустых значений

###### Удаляем.

text

In [5]:
df_articles = df_articles.dropna(subset='text')

In [6]:
nan_count = df_articles['text'].isna().sum().sum()
print('text: ', nan_count)

text:  0


authors

In [7]:
df_articles = df_articles.dropna(subset='authors')

In [8]:
nan_count = df_articles['authors'].isna().sum().sum()
print('authors: ', nan_count)

authors:  0


##### Заполняем пропуски.

annotation

In [9]:
df_articles['annotation'] = df_articles['annotation'].fillna('no annotation')

In [10]:
nan_count = df_articles['annotation'].isna().sum().sum()
print('annotation: ', nan_count)

annotation:  0


article_level

In [11]:
df_articles['article_level'] = df_articles['article_level'].fillna('-')

In [12]:
nan_count = df_articles['article_level'].isna().sum().sum()
print('article_level: ', nan_count)

article_level:  0


Итог:

In [13]:
df_articles.isnull().sum()

date             0
title            0
link             0
annotation       0
authors          0
article_level    0
text             0
dtype: int64

#### 3. Извлечение в отдельный столбец номера УДК

Сначала напишем регулярное выражение для извлечение УДК во всех форматах:

In [14]:
#создаём искусственный датафрейм со всеми подряд УДК в самых разных форматах
df = pd.DataFrame({ 'УДК': ['УДК 00.19 пыкаич', 'УДК 00.19\nпыкаич',
'яыпваУДК 9.1 пыкави',
'УДК 59 кпыаи',
'УДК: 00.59 пыкаич',
'УДК 378.001.76 пкыаи',
'﻿УДК 615.471:616.12 првптпе',
'АТКО УДК удкопрао удк 999 1-395']})

In [15]:
df

,УДК
0,УДК 00.19 пыкаич
1,УДК 00.19\nпыкаич
2,яыпваУДК 9.1 пыкави
3,УДК 59 кпыаи
4,УДК: 00.59 пыкаич
5,УДК 378.001.76 пкыаи
6,﻿УДК 615.471:616.12 првптпе
7,АТКО УДК удкопрао удк 999 1-395


In [16]:
#ищем совпадения
for el in df['УДК']:
    print(re.search(r'УДК:?\s+\d{1,3}(\.\d{1,3})?(.?|:?)?(\d{1,3}(\.\d{1,3})?)?', el)) 

<re.Match object; span=(0, 10), match='УДК 00.19 '>
<re.Match object; span=(0, 9), match='УДК 00.19'>
<re.Match object; span=(5, 13), match='УДК 9.1 '>
<re.Match object; span=(0, 7), match='УДК 59 '>
<re.Match object; span=(0, 11), match='УДК: 00.59 '>
<re.Match object; span=(0, 14), match='УДК 378.001.76'>
<re.Match object; span=(1, 19), match='УДК 615.471:616.12'>
None


In [17]:
#тестово делаем список
a = []
for el in df['УДК']:
    b = re.search(r'УДК:?\s+\d{1,3}(\.\d{1,3})?(.?|:?)?(\d{1,3}(\.\d{1,3})?)?', el)
    try:
        a.append(b.group(0))
    except AttributeError:
        a.append('None')
a

['УДК 00.19 ',
 'УДК 00.19',
 'УДК 9.1 ',
 'УДК 59 ',
 'УДК: 00.59 ',
 'УДК 378.001.76',
 'УДК 615.471:616.12',
 'None']

Сделаем то же самое для нашего датафрейма, и будем записывать результат в отдельный список.

In [18]:
udk = []
for el in df_articles['text']:
    reg = re.search(r'УДК:?\s+\d{1,3}(\.\d{1,3})?(.?|:?)?(\d{1,3}(\.\d{1,3})?)?', el)
    try:
        udk.append(reg.group(0))
    except AttributeError:
        udk.append('None')

In [19]:
udk[0:5]

['None', 'None', 'УДК 331.225.3', 'None', 'None']

In [20]:
len(udk)

2329

Теперь добавим новый столбец в датафрейм:

In [21]:
df_articles['UDK'] = udk

In [22]:
df_articles.head()

,date,title,link,annotation,authors,article_level,text,UDK
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,-,﻿GEOLOGICAL AND MINERALOGICAL SCIENCES\nTHE PR...,None
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",-,﻿TECHNICAL SCIENCES\nUNDERSTANDING AND DESIGNI...,None
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",-,﻿DEVELOPMENT OF AUTHENTICATION MODEL FOR CLOUD...,УДК 331.225.3
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,-,﻿DEVELOPMENT OF A MODEL OF INFORMATION FLOWS I...,None
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,-,﻿AUTOMATED CONTROL SYSTEM OF THE SOLAR AIR MAN...,None


Сохраняю датафрейм в файл эксель и просмотрю его глазами на предмет исключения ошибок:

In [23]:
df_articles.to_excel('date_with_udk2.xlsx', index=False)

#### 4. Уровень публикации переводим в OneHotEncoder

In [24]:
#для начала посмотрим сколько у нас есть уникальных комбинаций уровней статей
df_articles['article_level'].unique()

array(['-', 'WOS ВАК AGRIS', 'ВАК AGRIS', 'ВАК', 'Scopus ВАК ESCI',
       'ВАК RSCI', 'ВАК ESCI', 'ВАК CAS', 'Scopus ВАК RSCI',
       'Scopus ВАК RSCI ESCI', 'ВАК AGRIS RSCI', 'Scopus ВАК GeoRef',
       'Scopus ВАК', 'RSCI', 'ВАК CAS RSCI', 'Scopus',
       'Scopus ВАК CAS RSCI PubMed', 'ВАК zbMATH', 'AGRIS',
       'WOS Scopus ВАК', 'zbMATH RSCI ESCI ВАК MathSciNet',
       'WOS Scopus GeoRef', 'WOS ВАК RSCI', 'WOS', 'RSCI zbMATH'],
      dtype=object)

one hot encoder мог бы подойти, но, так как статьи могут принадлежать сразу многим уровням, стоит создать и заполнить новые столбцы вручную

In [25]:
#снова сначала попробуем на маленьком датафрейме где есть все наши значения
df = pd.DataFrame({ 'article_level': ['-', 'WOS ВАК AGRIS', 'ВАК AGRIS', 'ВАК', 'Scopus ВАК ESCI',
       'ВАК RSCI', 'ВАК ESCI', 'ВАК CAS', 'Scopus ВАК RSCI',
       'Scopus ВАК RSCI ESCI', 'ВАК AGRIS RSCI', 'Scopus ВАК GeoRef',
       'Scopus ВАК', 'RSCI', 'ВАК CAS RSCI', 'Scopus',
       'Scopus ВАК CAS RSCI PubMed', 'ВАК zbMATH', 'AGRIS',
       'WOS Scopus ВАК', 'zbMATH RSCI ESCI ВАК MathSciNet',
       'WOS Scopus GeoRef', 'WOS ВАК RSCI', 'WOS', 'RSCI zbMATH']})

In [26]:
df

,article_level
0,-
1,WOS ВАК AGRIS
2,ВАК AGRIS
3,ВАК
4,Scopus ВАК ESCI
5,ВАК RSCI
6,ВАК ESCI
7,ВАК CAS
8,Scopus ВАК RSCI
9,Scopus ВАК RSCI ESCI


In [44]:
#получаем список уникальных уровней публикации через множества
a = ' '.join(['-', 'WOS ВАК AGRIS', 'ВАК AGRIS', 'ВАК', 'Scopus ВАК ESCI',
       'ВАК RSCI', 'ВАК ESCI', 'ВАК CAS', 'Scopus ВАК RSCI',
       'Scopus ВАК RSCI ESCI', 'ВАК AGRIS RSCI', 'Scopus ВАК GeoRef',
       'Scopus ВАК', 'RSCI', 'ВАК CAS RSCI', 'Scopus',
       'Scopus ВАК CAS RSCI PubMed', 'ВАК zbMATH', 'AGRIS',
       'WOS Scopus ВАК', 'zbMATH RSCI ESCI ВАК MathSciNet',
       'WOS Scopus GeoRef', 'WOS ВАК RSCI', 'WOS', 'RSCI zbMATH'])
b = a.split(' ')
level = set(b)
list_level = list(list_level)
list_level

['ESCI',
 'GeoRef',
 'ВАК',
 '-',
 'RSCI',
 'WOS',
 'AGRIS',
 'PubMed',
 'MathSciNet',
 'Scopus',
 'zbMATH',
 'CAS']

In [55]:
#проверяем вхождение, есть ли какой-нибудь уровень публикации в ячейке, если да, то 1, если нет, 0
all_levels = []
for levels in df['article_level']:
    temp_list = []
    for level in list_level:
        if level in levels:
            temp_list.append(1)
        else:
            temp_list.append(0)
    all_levels.append(temp_list)
all_levels

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]]

In [74]:
#переворачиваю список, чтобы его прицепить к датафрейму. когда-нибудь я вспомню о матричных операциях...
for i in range(0, len(all_levels[0])):
    temp_list = []
    for one_vec in all_levels:
        temp_list.append(one_vec[i])
    df[list_level[i]] = temp_list
    print(temp_list)

[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [75]:
df

,article_level,ESCI,GeoRef,ВАК,-,RSCI,WOS,AGRIS,PubMed,MathSciNet,Scopus,zbMATH,CAS
0,-,0,0,0,1,0,0,0,0,0,0,0,0
1,WOS ВАК AGRIS,0,0,1,0,0,1,1,0,0,0,0,0
2,ВАК AGRIS,0,0,1,0,0,0,1,0,0,0,0,0
3,ВАК,0,0,1,0,0,0,0,0,0,0,0,0
4,Scopus ВАК ESCI,1,0,1,0,0,0,0,0,0,1,0,0
5,ВАК RSCI,0,0,1,0,1,0,0,0,0,0,0,0
6,ВАК ESCI,1,0,1,0,0,0,0,0,0,0,0,0
7,ВАК CAS,0,0,1,0,0,0,0,0,0,0,0,1
8,Scopus ВАК RSCI,0,0,1,0,1,0,0,0,0,1,0,0
9,Scopus ВАК RSCI ESCI,1,0,1,0,1,0,0,0,0,1,0,0


На маленьком датафрейме попробовали, теперь на большом:

In [76]:
all_levels = []
for levels in df_articles['article_level']:
    temp_list = []
    for level in list_level:
        if level in levels:
            temp_list.append(1)
        else:
            temp_list.append(0)
    all_levels.append(temp_list)

In [80]:
len(all_levels)

2329

In [81]:
#переворачиваю список, чтобы его прицепить к датафрейму. когда-нибудь я вспомню о матричных операциях...
for i in range(0, len(all_levels[0])):
    temp_list = []
    for one_vec in all_levels:
        temp_list.append(one_vec[i])
    df_articles[list_level[i]] = temp_list

In [83]:
df_articles.tail()

,date,title,link,annotation,authors,article_level,text,UDK,ESCI,GeoRef,ВАК,-,RSCI,WOS,AGRIS,PubMed,MathSciNet,Scopus,zbMATH,CAS
2335,2024,Модель пилота как средство валидации перспекти...,/article/n/model-pilota-kak-sredstvo-validatsi...,Учитывая растущую значимость челове-ческого фа...,"И И. Грешников, Л С. Куравский, С Д. Логачев, ...",ВАК,﻿Модель пилота как средство валидации перспект...,None,0,0,1,0,0,0,0,0,0,0,0,0
2336,2024,Сравнительный анализ платформ для цифровизации...,/article/n/sravnitelnyy-analiz-platform-dlya-t...,Приведен сравнительный анализ исполь-зуемых те...,"В В. Бритвина, А К. Агоштинью, Г П. Конюхова",ВАК,﻿Сравнительный анализ платформ для цифровизаци...,None,0,0,1,0,0,0,0,0,0,0,0,0
2337,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",ВАК,﻿Программный комплекс распределенного тестиров...,None,0,0,1,0,0,0,0,0,0,0,0,0
2338,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,ВАК,﻿í rpQm'ÓTQ\nISSN 2782-4543 (online) ISSN 1997...,None,0,0,1,0,0,0,0,0,0,0,0,0
2339,2023,СОЦИАЛЬНЫЕ СЕТИ: ПРАВОВОЕ РЕГУЛИРОВАНИЕ,/article/n/sotsialnye-seti-pravovoe-regulirovanie,В статье рассматривается основные понятия соци...,"Нуркасым Кылычбекович Аркабаев, Эрмек Мамытови...",-,﻿ОШ МАМЛЕКЕТТИК УНИВЕРСИТЕТИНИН ЖАРЧЫСЫ\nВЕСТН...,УДК: 004.738.5,0,0,0,1,0,0,0,0,0,0,0,0


Всё хорошо. Столбец article_level можно удалить.

In [84]:
del df_articles['article_level']

In [85]:
df_articles.head()

,date,title,link,annotation,authors,text,UDK,ESCI,GeoRef,ВАК,-,RSCI,WOS,AGRIS,PubMed,MathSciNet,Scopus,zbMATH,CAS
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,﻿GEOLOGICAL AND MINERALOGICAL SCIENCES\nTHE PR...,None,0,0,0,1,0,0,0,0,0,0,0,0
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",﻿TECHNICAL SCIENCES\nUNDERSTANDING AND DESIGNI...,None,0,0,0,1,0,0,0,0,0,0,0,0
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",﻿DEVELOPMENT OF AUTHENTICATION MODEL FOR CLOUD...,УДК 331.225.3,0,0,0,1,0,0,0,0,0,0,0,0
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,﻿DEVELOPMENT OF A MODEL OF INFORMATION FLOWS I...,None,0,0,0,1,0,0,0,0,0,0,0,0
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,﻿AUTOMATED CONTROL SYSTEM OF THE SOLAR AIR MAN...,None,0,0,0,1,0,0,0,0,0,0,0,0


#### 5. Обработка количества слов в одной статье  и удаление слишком маленьких статей

In [87]:
len(df_articles['text'][0].split()) #количество слов в первой статье

776

In [88]:
count_words = []
for el in df_articles['text']:
    count_words.append(len(el.split()))

In [89]:
#ну и сразу добавим и эту информацию в датафрейм
df_articles['count_words'] = count_words

Статьи где слов меньше 200 удалим (рекомендуемое количество слов в аннотации до 200, поэтому оставлять статьи которые короче теоретической аннотации смысла нет).

Смотрим количество таких текстов:

In [91]:
#наверняка есть готовый метод, но тут пару строк вручную писать
i = 0
for el in df_articles['count_words']:
    if el < 200:
        i+=1
print(i)

2


Удаляем:

In [92]:
df_articles = df_articles.drop(df_articles[df_articles['count_words'] < 200].index)

In [93]:
len(df_articles)

2327

In [94]:
df_articles

,date,title,link,annotation,authors,text,UDK,ESCI,GeoRef,ВАК,-,RSCI,WOS,AGRIS,PubMed,MathSciNet,Scopus,zbMATH,CAS,count_words
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,﻿GEOLOGICAL AND MINERALOGICAL SCIENCES\nTHE PR...,None,0,0,0,1,0,0,0,0,0,0,0,0,776
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",﻿TECHNICAL SCIENCES\nUNDERSTANDING AND DESIGNI...,None,0,0,0,1,0,0,0,0,0,0,0,0,1421
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",﻿DEVELOPMENT OF AUTHENTICATION MODEL FOR CLOUD...,УДК 331.225.3,0,0,0,1,0,0,0,0,0,0,0,0,2719
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,﻿DEVELOPMENT OF A MODEL OF INFORMATION FLOWS I...,None,0,0,0,1,0,0,0,0,0,0,0,0,2256
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,﻿AUTOMATED CONTROL SYSTEM OF THE SOLAR AIR MAN...,None,0,0,0,1,0,0,0,0,0,0,0,0,1132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2024,Модель пилота как средство валидации перспекти...,/article/n/model-pilota-kak-sredstvo-validatsi...,Учитывая растущую значимость челове-ческого фа...,"И И. Грешников, Л С. Куравский, С Д. Логачев, ...",﻿Модель пилота как средство валидации перспект...,None,0,0,1,0,0,0,0,0,0,0,0,0,4106
2336,2024,Сравнительный анализ платформ для цифровизации...,/article/n/sravnitelnyy-analiz-platform-dlya-t...,Приведен сравнительный анализ исполь-зуемых те...,"В В. Бритвина, А К. Агоштинью, Г П. Конюхова",﻿Сравнительный анализ платформ для цифровизаци...,None,0,0,1,0,0,0,0,0,0,0,0,0,4431
2337,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",﻿Программный комплекс распределенного тестиров...,None,0,0,1,0,0,0,0,0,0,0,0,0,3839
2338,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,﻿í rpQm'ÓTQ\nISSN 2782-4543 (online) ISSN 1997...,None,0,0,1,0,0,0,0,0,0,0,0,0,3449


In [96]:
df_articles.to_excel('date_with_udk2.xlsx', index=False)

#### 6. Приведение к нижнему регистру, лемматизация, удаление стоп-слов

Обработаем столбцы text и annotation.

In [97]:
m = pymorphy3.MorphAnalyzer()

In [99]:
mystopwords = stopwords.words('russian') + [ 'удк',
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
]
ru_words = re.compile("[А-Яа-яЁё]+")


def words_only(text):
    return " ".join(ru_words.findall(text))


def lemmatize(text, mystem=m):
    try:
        return " ".join([m.parse(w)[0].normal_form for w in text.split(' ')]).strip()  
    except:
        return " "

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

    
def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

In [100]:
%%time
df_articles.text = df_articles.text.apply(preprocess)

CPU times: total: 13min 7s
Wall time: 13min 7s


In [101]:
df_articles

,date,title,link,annotation,authors,text,UDK,ESCI,GeoRef,ВАК,-,RSCI,WOS,AGRIS,PubMed,MathSciNet,Scopus,zbMATH,CAS,count_words
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,проблема обработка данные георадиолокационный ...,None,0,0,0,1,0,0,0,0,0,0,0,0,776
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",проектирование современный высоконадёжный сеть...,None,0,0,0,1,0,0,0,0,0,0,0,0,1421
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",разработка аутентификационный модель облачный ...,УДК 331.225.3,0,0,0,1,0,0,0,0,0,0,0,0,2719
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,разработка модель информационный поток система...,None,0,0,0,1,0,0,0,0,0,0,0,0,2256
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,автоматизированный система управление солнечны...,None,0,0,0,1,0,0,0,0,0,0,0,0,1132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2024,Модель пилота как средство валидации перспекти...,/article/n/model-pilota-kak-sredstvo-validatsi...,Учитывая растущую значимость челове-ческого фа...,"И И. Грешников, Л С. Куравский, С Д. Логачев, ...",модель пилот средство валидация перспективный ...,None,0,0,1,0,0,0,0,0,0,0,0,0,4106
2336,2024,Сравнительный анализ платформ для цифровизации...,/article/n/sravnitelnyy-analiz-platform-dlya-t...,Приведен сравнительный анализ исполь-зуемых те...,"В В. Бритвина, А К. Агоштинью, Г П. Конюхова",сравнительный анализ платформа цифровизация го...,None,0,0,1,0,0,0,0,0,0,0,0,0,4431
2337,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",программный комплекс распределённый тестирован...,None,0,0,1,0,0,0,0,0,0,0,0,0,3839
2338,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,филологический наука вопрос теория практика вы...,None,0,0,1,0,0,0,0,0,0,0,0,0,3449


Ещё раз проверю статьи с количеством слов меньше 200 (мало ли что произошло при обработке):

In [102]:
#наверняка есть готовый метод, но тут пару строк вручную писать
i = 0
for el in df_articles['count_words']:
    if el < 200:
        i+=1
print(i)

0


##### Промежуточный вывод:
Я предобработала текст и извлекла всю полезную информацию которую смогла придумать без использования векторизации текста и дальнейшей интеллектуальной обработки (только pymorphy использовала для лемматизации). В 3 части я попробую посмотреть на закономерности и статистики такого датафрейма, после чего проведу векторизацию и начну работать непосредственно с текстами статей.